In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/Github/Deceptive-Review-Detection'

/content/drive/MyDrive/Github/Deceptive-Review-Detection


In [4]:
# pip install transformers

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW

from transformers import BertTokenizer, BertModel, BertConfig
import sys

In [6]:
import data_prep
from BERT_Classification import *

In [7]:
# df = data_prep.import_chi_review()
df = pd.read_csv("Chicago_Hotel_Review/Chicago_Hotel_Reviews.csv", index_col=0)
df = df.drop('Rating', axis=1)
df.head()

,Label,Ori_Review
0,1,Affinia Chicago is one of the worst hotels I h...
1,1,I recently stayed at the Affina Chicago hotel ...
2,1,I stayed at the Affina Chicago for my annivers...
3,1,If you are looking for a high end hotel on the...
4,1,I just returned from a long weekend in Chicago...


In [8]:
X = df['Ori_Review']
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
X_train = X_train.to_list()
X_test = X_test.to_list()
X_val = X_val.to_list()
y_train = y_train.to_list()
y_test = y_test.to_list()
y_val = y_val.to_list()

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device used: {}.".format(device))

Device used: cuda.


In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
train_dataset = dataset(tokenizer, X_train, y_train, max_seq_length=512)
val_dataset = dataset(tokenizer, X_val, y_val, max_seq_length=512)
test_dataset = dataset(tokenizer, X_test, y_test, max_seq_length=512)

In [12]:
train_data_loader = DataLoader(dataset=train_dataset, 
                         batch_size=8, 
                         shuffle=True)
val_data_loader = DataLoader(dataset=val_dataset, 
                         batch_size=8, 
                         shuffle=True)
test_data_loader = DataLoader(dataset=test_dataset, 
                         batch_size=8, 
                         shuffle=True)

In [13]:
model = BertWithClassifier(bert, 512, nn.CrossEntropyLoss).to(device)

In [ ]:
train_losses, val_losses = train(model=model, 
                                data_loader=train_data_loader, 
                                val_data_loader=val_data_loader, 
                                optimizer=AdamW, 
                                learning_rate=1e-5,
                                loss_fn=nn.CrossEntropyLoss,
                                num_epochs=5,
                                device=device)

training:   0%|          | 0/5 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/162 [00:00<?, ?batch/s]

tensor(0.5347, device='cuda:0')


epoch 2:   0%|          | 0/162 [00:00<?, ?batch/s]

tensor(0.4722, device='cuda:0')


epoch 3:   0%|          | 0/162 [00:00<?, ?batch/s]

In [ ]:
test_preds, test_loss, test_acc = validate(model, test_data_loader, device)

In [ ]:
test_acc